# Fake News Classifier using Bidirectional LSTM RNN...

In [1]:
import pandas as pd
df = pd.read_csv('train.csv', quoting=1, on_bad_lines='skip', engine='python')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [2]:
df.isnull().sum()

id          0
title     162
author    582
text       12
label       0
dtype: int64

In [3]:
df.shape

(5975, 5)

In [4]:
df = df.dropna()

In [5]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [7]:
df.shape

(5231, 5)

In [10]:
# Separating the dependent and independent features...
X = df.drop(['label'], axis=1)
y = df['label']

In [11]:
# Check weather the data is balanced or imbalanced...
y.value_counts()

label
0    2967
1    2264
Name: count, dtype: int64

In [12]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [13]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [14]:
## vocabulary size...
voc_size = 5000

In [15]:
messages = X.copy()

In [22]:
messages.head(8)

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
6,7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
7,9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...


In [17]:
import nltk
import re
from nltk.corpus import stopwords

In [18]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
messages.reset_index(inplace=True)

In [21]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [23]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [24]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[2099, 4900, 1362, 1697, 2595, 3323, 3885, 4678, 4260, 1864],
 [1257, 4170, 302, 769, 187, 1507, 1073],
 [1521, 2574, 1834, 1148],
 [4130, 1708, 3148, 1778, 1055, 3086],
 [2398, 187, 4034, 4237, 2911, 354, 187, 265, 1062, 2989],
 [3305,
  2684,
  373,
  4253,
  1608,
  2888,
  607,
  4026,
  1669,
  4389,
  1726,
  1081,
  2382,
  315,
  1073],
 [4366, 3899, 1009, 3550, 441, 3126, 4860, 1467, 3198, 1500, 310],
 [1504, 889, 1649, 2258, 1207, 4339, 2888, 309, 3198, 1500, 310],
 [711, 1198, 2333, 1218, 3024, 4686, 4576, 4398, 2888, 124],
 [4359, 3924, 4086, 1419, 1253, 2563, 3983, 2058],
 [2573, 45, 3811, 4089, 561, 2343, 4760, 1344, 3989, 3400, 2482],
 [1778, 4163, 2595, 4686, 2888, 1207],
 [4543, 518, 4425, 1245, 4709, 1421, 3596, 2340, 2652],
 [2104, 3122, 2872, 1588, 919, 1122, 2707, 3198, 1500, 310],
 [4374, 3964, 595, 3752, 4591, 3198, 1500, 310],
 [2870, 1010, 295, 2672, 4811, 1361, 3386, 545, 3475, 285],
 [720, 2357, 4170],
 [916, 4402, 3221, 1069, 2888, 513, 41, 1073],
 [1272, 2

### Embedding Representation

In [25]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 4678, 4260, 1864],
       [   0,    0,    0, ...,  187, 1507, 1073],
       [   0,    0,    0, ..., 2574, 1834, 1148],
       ...,
       [   0,    0,    0, ...,   89, 1515, 1073],
       [   0,    0,    0, ..., 2726,  762, 1073],
       [   0,    0,    0, ..., 1708, 4130, 4267]], dtype=int32)

In [26]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100))) ## Note that we can play with different no. of neurons in a LSTM...
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [27]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [28]:
## train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [29]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
110/110 [==============================] - 9s 39ms/step - loss: 0.3963 - accuracy: 0.8054 - val_loss: 0.2516 - val_accuracy: 0.9045
Epoch 2/10
110/110 [==============================] - 4s 33ms/step - loss: 0.1457 - accuracy: 0.9458 - val_loss: 0.2252 - val_accuracy: 0.9131
Epoch 3/10
110/110 [==============================] - 5s 42ms/step - loss: 0.0681 - accuracy: 0.9806 - val_loss: 0.3185 - val_accuracy: 0.9102
Epoch 4/10
110/110 [==============================] - 3s 30ms/step - loss: 0.0318 - accuracy: 0.9923 - val_loss: 0.3518 - val_accuracy: 0.8964
Epoch 5/10
110/110 [==============================] - 4s 32ms/step - loss: 0.0190 - accuracy: 0.9954 - val_loss: 0.4844 - val_accuracy: 0.8975
Epoch 6/10
110/110 [==============================] - 4s 37ms/step - loss: 0.0091 - accuracy: 0.9977 - val_loss: 0.5219 - val_accuracy: 0.8923
Epoch 7/10
110/110 [==============================] - 5s 43ms/step - loss: 0.0077 - accuracy: 0.9977 - val_loss: 0.5461 - val_accuracy: 0.8998

In [31]:
y_pred = model.predict(X_test)

54/54 [==============================] - 1s 7ms/step


In [34]:
import numpy as np
y_pred=np.where(y_pred>=0.5,1,0)

In [37]:
y_pred[0]

array([1])

In [38]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[922,  75],
       [113, 617]])

In [39]:
print(accuracy_score(y_test,y_pred))

0.8911407064273307
